In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import open3d as o3d
from torch.utils.data import Dataset, DataLoader, Subset
from dataclasses import dataclass
import sys
from pathlib import Path
from pytorch3d.loss import chamfer_distance
from tqdm import tqdm

sys.path.append(str(Path.cwd().parent))

import AutoEncoder.autoencoders as ae
from AutoEncoder.encoder import PointCloudAutoEncoder
from Helpers.data import PointCloudDataset
import Helpers.PointCloudOpen3d as pc
from Helpers.nsdf_live_sampler import sample_sdf
from SignDistanceModel.sdf_models import SDFRegressionModel

if torch.cuda.is_available():
    device = "cuda"

elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f'Using: {device}')


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using: cuda


In [2]:
train_dataset_1024 = PointCloudDataset("../Data/ModelNet40", 1024, 'train', object_classes =  ["sofa"] )
test_dataset_1024 = PointCloudDataset("../Data/ModelNet40", 1024, 'test', object_classes =  ["sofa"])

test_size = len(test_dataset_1024)
split_idx = test_size // 2
indices = list(range(test_size))
val_dataset_1024 = Subset(test_dataset_1024, indices[:split_idx])
test_dataset_1024 = Subset(test_dataset_1024, indices[split_idx:])

train_loader_1024= DataLoader(train_dataset_1024, batch_size = 64, shuffle = True)
val_loader_1024 = DataLoader(val_dataset_1024, batch_size = 128, shuffle = False)
test_loader_1024 = DataLoader(test_dataset_1024, batch_size = 128, shuffle = False)

In [26]:
train_dataset_3072 = PointCloudDataset("../Data/ModelNet40", 3072, 'train', object_classes = None )
test_dataset_3072 = PointCloudDataset("../Data/ModelNet40", 3072, 'test', object_classes = None)

test_size = len(test_dataset_3072)
split_idx = test_size // 2
indices = list(range(test_size))
val_dataset_3072 = Subset(test_dataset_3072, indices[:split_idx])
test_dataset_3072 = Subset(test_dataset_3072, indices[split_idx:])

train_loader_3072 = DataLoader(train_dataset_3072, batch_size = 64, shuffle = True)
val_loader_3072 = DataLoader(val_dataset_3072, batch_size = 128, shuffle = False)
test_loader_3072 = DataLoader(test_dataset_3072, batch_size = 128, shuffle = False)

In [3]:
torch.save(train_dataset_1024 , 'train_dataset_1024.pt')
torch.save(test_dataset_1024, 'test_dataset_1024.pt')
torch.save(val_dataset_1024, 'val_dataset_1024.pt')

In [27]:
torch.save(train_dataset_3072 , 'train_dataset_3072.pt')
torch.save(test_dataset_3072, 'test_dataset_3072.pt')
torch.save(val_dataset_3072, 'val_dataset_3072.pt')

In [ ]:
val_dataset_1024 = torch.load('val_dataset_1024.pt', weights_only = False)
test_dataset_1024 = torch.load('test_dataset_1024.pt', weights_only = False)
train_dataset_1024 = torch.load('train_dataset_1024.pt', weights_only = False)

train_loader_1024= DataLoader(train_dataset_1024, batch_size = 64, shuffle = True)
val_loader_1024 = DataLoader(val_dataset_1024, batch_size = 64, shuffle = False)
test_loader_1024 = DataLoader(test_dataset_1024, batch_size = 64, shuffle = False)

In [18]:
val_dataset_3072 = torch.load('val_dataset_3072.pt', weights_only = False)
test_dataset_3072 = torch.load('test_dataset_3072.pt', weights_only = False)
train_dataset_3072 = torch.load('train_dataset_3072.pt', weights_only = False)

train_loader_3072 = DataLoader(train_dataset_3072, batch_size = 64, shuffle = True)
val_loader_3072 = DataLoader(val_dataset_3072, batch_size = 64, shuffle = False)
test_loader_3072 = DataLoader(test_dataset_3072, batch_size = 64, shuffle = False)

In [3]:
# Load model
model = ae.ConvAE_6800T(1024, 256)
model.load_state_dict(torch.load('../Data/trained_autoencoders/1024_256/Conv_6800T', weights_only=True))
print(model.to('cpu'))
model = model.to('cpu')

ConvAE_6800T(
  (encoder): ConvEncoder_6800T(
    (conv1): Conv1d(3, 32, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(32, 32, kernel_size=(9,), stride=(1,), padding=(4,))
    (conv3): Conv1d(32, 32, kernel_size=(9,), stride=(1,), padding=(4,))
    (conv4): Conv1d(32, 16, kernel_size=(8,), stride=(2,), padding=(3,))
    (conv5): Conv1d(16, 16, kernel_size=(8,), stride=(2,), padding=(3,))
    (conv6): Conv1d(16, 16, kernel_size=(8,), stride=(2,), padding=(3,))
    (lin1): Linear(in_features=2048, out_features=1024, bias=True)
    (lin2): Linear(in_features=1024, out_features=256, bias=True)
  )
  (decoder): ConvDecoder(
    (l1): Linear(in_features=256, out_features=1024, bias=True)
    (l2): Linear(in_features=1024, out_features=2048, bias=True)
    (l3): Linear(in_features=2048, out_features=3072, bias=True)
    (l4): Linear(in_features=3072, out_features=3072, bias=True)
  )
)


In [4]:
def visualize_random_reconstruction(model, x, device, transpose_out = False, show_org = True):
    '''
    Pick a random cloud from the dataset and show what it looks like before and after autoencoder reconstruction 
    First window is original point cloud 
    Second window is recontructed point cloud
    '''

    def show_cloud(x):
        pc.visualize_point_cloud(pc.get_point_cloud(x.cpu().numpy().astype(np.float64)))

    
    if show_org:
        show_cloud(x)

    with torch.no_grad():
        x = x.unsqueeze(0).permute(0,2,1).to(device)
        
        rec_x = model(x)[0].to('cpu')
        
        if transpose_out:
            rec_x = rec_x.T
        
        rec_x = rec_x

        show_cloud(rec_x)

#visualize_random_reconstruction(model, x, device)

In [5]:
x = next(iter(val_loader_1024))
visualize_random_reconstruction(model, x['points'][0].to('cpu'), 'cpu')
print(x["filename"][0])

../Data/ModelNet40/sofa/test/sofa_0706.off


In [5]:
def collect_points_batch(point_cloud, encoder_model, num_points=1000, batches=64):
    sampled_points = None
    sampled_labels = None
    corr_neural_encoding = None
    filenames = point_cloud["filename"]
    for i in range(len(filenames)):
        if batches <= i:
            break
        filename_split = filenames[i].split('/')
        filename_split[2] = "vdbs"
        filename_split[-1]  = filename_split[-1][:-3] + "vdb" 
        vdb_filename = "".join([i + '/' for i in filename_split])[:-1]
        points, labels = sample_sdf(vdb_filename, num_points, device=device)
        x = point_cloud['points'].to(device)[i].unsqueeze(0)
        y = x.permute(0,2,1)
        with torch.no_grad():
            neural_encoding = encoder_model.encoder(y)
        if sampled_points == None:
            sampled_points = points
            sampled_labels = labels
            corr_neural_encoding = neural_encoding.repeat(num_points, 1)
        else:
            sampled_points = torch.cat((sampled_points, points), dim=0)
            sampled_labels = torch.cat((sampled_labels, labels), dim=0)
            corr_neural_encoding = torch.cat((corr_neural_encoding, neural_encoding.repeat(num_points, 1)))
    return sampled_points, sampled_labels, corr_neural_encoding

def train_one_mesh(filename, model, neural_encoding, optimizer, live_sampling=True, num_points=1000, num_epochs = 10):
    train_losses = []
    val_losses = []
    criterion = nn.MSELoss()
    filename_split = filename.split('/')
    filename_split[2] = "vdbs"
    filename_split[-1]  = filename_split[-1][:-3] + "vdb" 
    vdb_filename = "".join([i + '/' for i in filename_split])[:-1]
    neural_encoding = neural_encoding.repeat(num_points, 1)
    running_acc = 0
    running_loss = 0
    for epoch in range(num_epochs):
        points, labels = sample_sdf(vdb_filename, num_points, device=device)
        perm = torch.randperm(num_points)
        points, labels = points[perm], labels[perm]
        labels[labels < 0] = -1
        labels[labels > 0] = 1
        optimizer.zero_grad()
        #print(points.shape)
        outputs = model(points, neural_encoding)
        loss = criterion(outputs, labels)
        loss.backward()
        #optimizer.step()
        label_signs = torch.sign(labels)
        output_signs = torch.sign(outputs)
        correct_signs = (label_signs == output_signs).sum().item()
        running_acc += correct_signs / num_points
        running_loss += loss.item()
    return running_loss / num_epochs, running_acc / num_epochs
        

In [12]:
x = next(iter(val_loader_1024))


In [31]:
class SDFRegressionModel(nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim):
        super(SDFRegressionModel, self).__init__()
        self.x_fc = nn.Linear(input_dim, hidden_dim)
        self.laten_fc = nn.Linear(latent_dim, hidden_dim) 
        self.batch_norm1 = nn.BatchNorm1d(2* hidden_dim, affine=True)
        self.fc1 = nn.Linear(2* hidden_dim, 2*hidden_dim)
        self.batch_norm2 = nn.BatchNorm1d(2*hidden_dim, affine=True)
        self.fc2 = nn.Linear(2*hidden_dim, hidden_dim)
        self.batch_norm3 = nn.BatchNorm1d(hidden_dim, affine=True)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.batch_norm4 = nn.BatchNorm1d(hidden_dim, affine=True)
        self.fc4 = nn.Linear(hidden_dim, 1)
        self.relu = nn.ReLU()


    def forward(self, x, latent_encoding):
        x = self.relu(self.x_fc(x))
        latent_encoding = self.relu(self.laten_fc(latent_encoding))
        x = torch.cat((x, latent_encoding), dim=-1)
        x = self.batch_norm1(x)
        x = self.batch_norm2(self.relu(self.fc1(x)))
        x = self.batch_norm3(self.relu(self.fc2(x)))
        x = self.batch_norm4(self.relu(self.fc3(x)))
        x = self.fc4(x)
        return x


In [ ]:
sdf_model = SDFRegressionModel(3, 256, 256)
#sdf_model = SDFRegressionModel(3, 256, 64)
#sdf_model.load_state_dict(torch.load("sdf_model_1024_256.pth"))
sdf_model.to(device)
#sdf_model = my_model
#sdf_model.eval()
optimizer = optim.AdamW(sdf_model.parameters(), lr=0.001)
criterion = nn.MSELoss()
model.to(device)
for i in range(50): 
    counter = 0
    sum_correct_signs = 0
    for point_cloud in train_loader_1024:
        counter += 1
        points, labels, neural_encoding = collect_points_batch(point_cloud, model, num_points=2, batches=64)
        perm = torch.randperm(points.shape[0])
        points, labels, neural_encoding = points[perm], labels[perm], neural_encoding[perm]
        labels[labels < 0] = -1
        labels[labels > 0] = 1
        optimizer.zero_grad()
        outputs = sdf_model(points, neural_encoding)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        label_signs = torch.sign(labels)
        output_signs = torch.sign(outputs)
        correct_signs = (label_signs == output_signs).sum().item()
        sum_correct_signs += correct_signs
        #print(correct_signs)
        #print(f'{i} Loss: {loss.item():.4f} {len(point_cloud["filename"])}')
    print(sum_correct_signs / (counter * 2 * 1))
    my_model = sdf_model.to('cpu')
    torch.save(my_model.state_dict(), "sdf_model_1024_256.pth")    
    sdf_model.to(device)
    

40.76623376623377
45.422077922077925
47.61363636363637
48.87337662337662
49.87987012987013
50.86363636363637
51.506493506493506
52.51948051948052
